In [1]:
!nvidia-smi

Sun Dec 28 15:26:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.80                 Driver Version: 581.80         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   51C    P8              2W /  160W |     198MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from pathlib import Path

HOME = Path.cwd()
print("HOME:", HOME)

HOME: c:\Users\kevor\Documents\ML_journey\random_code\basketball


In [3]:
import os, subprocess, sys
from pathlib import Path

def load_msvc_env(arch="x64"):
    """
    Loads Visual Studio MSVC build environment into the *current* process env.
    Works from Jupyter/Anaconda without needing to launch a special prompt.
    """
    if os.name != "nt":
        return

    vswhere = Path(os.environ.get("ProgramFiles(x86)", r"C:\Program Files (x86)")) / \
              "Microsoft Visual Studio" / "Installer" / "vswhere.exe"
    if not vswhere.exists():
        raise FileNotFoundError(f"vswhere.exe not found at: {vswhere}")

    # Find a VS install that has C++ tools
    install_path = subprocess.check_output([
        str(vswhere),
        "-latest", "-products", "*",
        "-requires", "Microsoft.VisualStudio.Component.VC.Tools.x86.x64",
        "-property", "installationPath"
    ], text=True).strip()

    if not install_path:
        raise RuntimeError(
            "vswhere found no VS install with MSVC C++ tools.\n"
            "Open Visual Studio Installer → Modify → install 'Desktop development with C++' "
            "and MSVC v143 + Windows SDK."
        )

    vcvars = Path(install_path) / "VC" / "Auxiliary" / "Build" / "vcvars64.bat"
    if not vcvars.exists():
        # fallback to vcvarsall
        vcvars = Path(install_path) / "VC" / "Auxiliary" / "Build" / "vcvarsall.bat"
        if not vcvars.exists():
            raise FileNotFoundError(f"Could not find vcvars script under: {install_path}")

    # Run vcvars and capture the environment it sets
    cmd = f'"{vcvars}" {arch} && set'
    output = subprocess.check_output(cmd, shell=True, text=True)

    # Apply captured env vars to this process
    for line in output.splitlines():
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k] = v

    # Sanity check
    where_cl = subprocess.check_output("where cl", shell=True, text=True).strip()
    print("MSVC initialized. cl is at:\n", where_cl)

load_msvc_env("x64")


MSVC initialized. cl is at:
 C:\Program Files\Microsoft Visual Studio\18\Community\VC\Tools\MSVC\14.50.35717\bin\Hostx64\x64\cl.exe


In [4]:
!where cl
!where link
!where rc


C:\Program Files\Microsoft Visual Studio\18\Community\VC\Tools\MSVC\14.50.35717\bin\Hostx64\x64\cl.exe
C:\Program Files\Microsoft Visual Studio\18\Community\VC\Tools\MSVC\14.50.35717\bin\Hostx64\x64\link.exe
C:\Program Files (x86)\Windows Kits\10\bin\10.0.26100.0\x64\rc.exe


In [5]:
import os
os.environ["VERBOSE"] = "1"
os.environ["MAX_JOBS"] = "1"

# prevents distutils trying to activate VC env again
os.environ["DISTUTILS_USE_SDK"] = "1"
os.environ["MSSdk"] = "1"

# Try this FIRST to get a clearer, more reliable build on Windows
os.environ["USE_NINJA"] = "0"



In [6]:
import os
import subprocess
from pathlib import Path

# Clone the repository (skip if already exists)
repo_dir = HOME / "segment-anything-2-real-time"
if not repo_dir.exists():
    subprocess.run(["git", "clone", "https://github.com/Gy920/segment-anything-2-real-time.git"], 
                  cwd=str(HOME), check=True)
    print("Repository cloned successfully")
else:
    print(f"Repository already exists at {repo_dir}")

# Change to the cloned directory
os.chdir(str(repo_dir))

# Install numpy first (required for build process)
!pip install numpy -q

# Try to detect and set CUDA_HOME on Windows
if os.name == 'nt':  # Windows
    cuda_home = os.environ.get('CUDA_HOME') or os.environ.get('CUDA_PATH')
    
    # If not set, try common installation paths
    if not cuda_home:
        common_cuda_paths = [
            # r'C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0',
            r'C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1',
            # r'C:\Program Files (x86)\NVIDIA GPU Computing Toolkit\CUDA\v13.0',
            r'C:\Program Files (x86)\NVIDIA GPU Computing Toolkit\CUDA\v12.1',
        ]
        
        for path in common_cuda_paths:
            if Path(path).exists():
                cuda_home = path
                break
    
    if cuda_home:
        os.environ['CUDA_HOME'] = cuda_home
        os.environ['CUDA_PATH'] = cuda_home
        print(f"Set CUDA_HOME to: {cuda_home}")
    else:
        print("WARNING: CUDA_HOME not found. Please install CUDA Toolkit and set CUDA_HOME environment variable.")
        print("Common locations: C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\vX.X")
    
    # Check for Visual Studio Build Tools
    vs_paths = [
        r'C:\Program Files\Microsoft Visual Studio\2022\BuildTools',
        r'C:\Program Files\Microsoft Visual Studio\2022\Community',
        r'C:\Program Files\Microsoft Visual Studio\2022\Professional',
        r'C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools',
    ]
    
    vs_found = any(Path(p).exists() for p in vs_paths)
    if not vs_found:
        print("WARNING: Visual Studio Build Tools not found.")
        print("Please install Visual Studio Build Tools with C++ compiler support.")
        print("Download from: https://visualstudio.microsoft.com/downloads/")

# Clean previous build artifacts (important for Windows)
import shutil
build_dirs = ["build", "dist", "*.egg-info"]
for pattern in build_dirs:
    for path in repo_dir.glob(pattern):
        if path.is_dir():
            shutil.rmtree(path, ignore_errors=True)
            print(f"Cleaned: {path}")

# Install the package
print("Installing package...")
print("Note: setup.py has been modified to support Visual Studio 2022 with CUDA 12.1")
!pip install -e . --no-build-isolation

# Build extensions (if needed)
print("Building extensions...")
!python setup.py build_ext --inplace

Repository already exists at c:\Users\kevor\Documents\ML_journey\random_code\basketball\segment-anything-2-real-time
Set CUDA_HOME to: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1
Cleaned: c:\Users\kevor\Documents\ML_journey\random_code\basketball\segment-anything-2-real-time\build
Installing package...
Note: setup.py has been modified to support Visual Studio 2022 with CUDA 12.1
Obtaining file:///C:/Users/kevor/Documents/ML_journey/random_code/basketball/segment-anything-2-real-time
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached omegaconf-2.3.0-py3-none-any.

  error: subprocess-exited-with-error
  
  × Building editable for SAM-2 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [108 lines of output]
      running editable_wheel
      creating C:\Users\kevor\AppData\Local\Temp\pip-ephem-wheel-cache-us6sxyyy\wheels\23\77\fc\5b2926fd18c2a3191c459ef49f5d5b2efcfcd75514e6f277f8\tmpn00tgoy7\.tmp-fo70m1sm\SAM_2.egg-info
      writing C:\Users\kevor\AppData\Local\Temp\pip-ephem-wheel-cache-us6sxyyy\wheels\23\77\fc\5b2926fd18c2a3191c459ef49f5d5b2efcfcd75514e6f277f8\tmpn00tgoy7\.tmp-fo70m1sm\SAM_2.egg-info\PKG-INFO
      writing dependency_links to C:\Users\kevor\AppData\Local\Temp\pip-ephem-wheel-cache-us6sxyyy\wheels\23\77\fc\5b2926fd18c2a3191c459ef49f5d5b2efcfcd75514e6f277f8\tmpn00tgoy7\.tmp-fo70m1sm\SAM_2.egg-info\dependency_links.txt
      writing requirements to C:\Users\kevor\AppData\Local\Temp\pip-ephem-wheel-cache-us6sxyyy\wheels\23\77\fc\5b2926fd18c2a3191c459ef49f5d5b2efcfcd75514e6f277f8\tmpn00tgoy7\.tmp-fo70m1sm\SAM_2.egg-

running build_ext
building 'sam2._C' extension
creating c:\Users\kevor\Documents\ML_journey\random_code\basketball\segment-anything-2-real-time\build\temp.win-amd64-cpython-310\Release\sam2\csrc
[1/1] C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin\nvcc --generate-dependencies-with-compile --dependency-output c:\Users\kevor\Documents\ML_journey\random_code\basketball\segment-anything-2-real-time\build\temp.win-amd64-cpython-310\Release\sam2\csrc\connected_components.obj.d -std=c++17 --use-local-env -Xcompiler /MD -Xcompiler /wd4819 -Xcompiler /wd4251 -Xcompiler /wd4244 -Xcompiler /wd4267 -Xcompiler /wd4275 -Xcompiler /wd4018 -Xcompiler /wd4190 -Xcompiler /wd4624 -Xcompiler /wd4067 -Xcompiler /wd4068 -Xcompiler /EHsc -Xcudafe --diag_suppress=base_class_has_different_dll_interface -Xcudafe --diag_suppress=field_without_dll_interface -Xcudafe --diag_suppress=dll_interface_conflict_none_assumed -Xcudafe --diag_suppress=dll_interface_conflict_dllexport_assumed -Ic:\Users\kevor\

c:\Users\kevor\anaconda3\envs\roboflow\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Emitting ninja build file c:\Users\kevor\Documents\ML_journey\random_code\basketball\segment-anything-2-real-time\build\temp.win-amd64-cpython-310\Release\build.ninja...
Compiling objects...
Using envvar MAX_JOBS (1) as the number of workers...
Traceback (most recent call last):
  File "c:\Users\kevor\anaconda3\envs\roboflow\lib\site-packages\torch\utils\cpp_extension.py", line 2104, in _run_ninja_build
    subprocess.run(
  File "c:\Users\kevor\anaconda3\envs\roboflow\lib\subprocess.py", line 526, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ninja', '-v', '-j', '1']' returned non-zero exit status 1.

The above exception was the direct cause of the fo